In [1]:
from sympy import *
import numpy as np
import quaternion
from DCM_sym import HEB
#init_printing()


Given that $q$ is a quaternion representation of the vehicles orientation and a vector is represented as a quaternion where the real part is zero and the i,j,k components are the vector (`quaternion.Quaternion.from_vec(some_vector)`).

To convert from inertial to body coordinate system:
$$
v_{body} = q^{-1} v_{inertial} q
$$
and to convert from body to inertial coordinate system:
$$
v_{inertial} = q v_{body} q^{-1}
$$

| State Var | Description            | Units   |
|:-----------|:------------------------|:---------:|
| $q_0$      | Quaternion    |  |
| $q_1$      | Quaternion   |  |
| $q_2$      | Quaternion     |  |
| $q_3$      | Quaternion     |  |
| $a_x$     | Body axis acceleration | m/s$^2$ |
| $a_y$     | Body axis acceleration | m/s$^2$ |
| $a_z$     | Body axis acceleration | m/s$^2$ |
| $w_x$     | Body axis rotation rate | rad/s |
| $w_y$     | Body axis rotation rate | rad/s |
| $w_z$     | Body axis rotation rate | rad/s |
| $TAS$     | True airspeed          | m/s |

In [2]:
q0, q1, q2, q3 = symbols(['q0', 'q1', 'q2', 'q3'])
wx, wy, wz = symbols(['wx', 'wy', 'wz'])
ax, ay, az = symbols(['ax', 'ay', 'az'])
tas = symbols('tas')
state = [q0, q1, q2, q3, ax, ay, az, wx, wy, wz, tas]

g = symbols('g')
dt = symbols('dt')

The model used for this filter is an assumption of coordinated flight.  Using this assumption, we can calculate the [rate of rotation](https://en.wikipedia.org/wiki/Standard_rate_turn) (about inertial z axis) and [centripital acceleration](https://en.wikipedia.org/wiki/Centripetal_force) due to this rotation.  

Centripital acceleration derivation by considering the triangle of a banked airplane with one leg of centripital acceleration and the other gravity.
$$ \tan \phi = \frac{a_c}{g} $$
where $\phi$ is the angle of bank.  $a_c=v^2/r$ so this equation can also be expressed as:
$$ \tan \phi = \frac{v^2}{r g} $$

Rate of rotation derivation. The aircraft completes a full rotation of a circle of radius $r$ in $2\pi r/v$ seconds.  Therefore, the rate of rotation (in inertial coordinates) can be written as:
$$ \dot{\psi} = \frac{2 \pi v}{2\pi r} = \frac{v}{r} = a_c/v $$
The rate of rotation simplifies to:
$$ \dot{\psi} = \frac{g}{v} \tan \phi $$

We will use the [Euler angles](https://en.wikipedia.org/wiki/Euler_angles#Tait%E2%80%93Bryan_angles) (more properly, they are the Tait-Bryan convention) for $\phi$ in the above equations as well as to create an intermediate coordinate system where the $y$ axis is the direction of the centripetal acceleration.

In [3]:
quaternion.arctan2 = atan2  # Make quaternion use symbolic functions
quaternion.arcsin = asin
quaternion.sqrt = sqrt
q = quaternion.Quaternion(q0,q1,q2,q3)
print("Roll from quaternion:", q.euler_angles()[0])
print("Heading from quaternion:", q.euler_angles()[2])

Roll from quaternion: atan2(2*q0*q1 + 2*q2*q3, -2*q1**2 - 2*q2**2 + 1)
Heading from quaternion: atan2(2*q0*q3 + 2*q1*q2, -2*q2**2 - 2*q3**2 + 1)


We now have enought information to start working through the Kalman filter update equations.

In [29]:
# Couple options here:
# 1) Propagate by current state rotations with very low Kalman Q components
# 2) Assume coordinated flight with higher Kalman Q components
qdelta = quaternion.Quaternion(1, wx/2*dt, wy/2*dt, wz/2*dt)  # small angle approximation of rotation
qnext = quaternion.Quaternion(q0,q1,q2,q3)*qdelta   # Option 1

# Option 2
phi = atan2(2*q0*q1 + 2*q2*q3, -2*q1**2 - 2*q2**2 + 1)
psi = -g/tas*tan(phi)
#qnext = quaternion.Quaternion(1, 0, 0, .5*psi*dt)*quaternion.Quaternion(q0,q1,q2,q3)
#qnext.normalize()

ag = np.array([0,0,g])  


# vector, in inertial coords, in direction which ac acts
v_ac = (q*quaternion.Quaternion.from_vec([0,-1.0,0])*q.inv()).as_ndarray()[1:]
v_ac[2] = 0
v_ac /= sqrt(v_ac[0]**2 + v_ac[1]**2)

ac = v_ac*g*tan(phi)
a = ag + ac

ab = (q.inv()*quaternion.Quaternion.from_vec(a)*q).as_ndarray()[1:]

wpredict = (q.inv()*quaternion.Quaternion.from_vec([0,0,-g/tas*tan(phi)])*q).as_ndarray()[1:]
statenext = np.concatenate([qnext.as_ndarray(), ab, wpredict, [tas + ax*dt]])
statenext = np.array([x.subs(q0**2+q1**2+q2**2+q3**2,1) for x in statenext])
for n in range(len(state)):
    print(f"{state[n]} = {statenext[n]}")

q0 = -dt*q1*wx/2 - dt*q2*wy/2 - dt*q3*wz/2 + q0
q1 = dt*q0*wx/2 + dt*q2*wz/2 - dt*q3*wy/2 + q1
q2 = dt*q0*wy/2 - dt*q1*wz/2 + dt*q3*wx/2 + q2
q3 = dt*q0*wz/2 + dt*q1*wy/2 - dt*q2*wx/2 + q3
ax = q0*(0.5*g*q0*(2*q0*q1 + 2*q2*q3)*(2.0*q0*q3 - 2.0*q1*q2)/(sqrt((q0*q3 - q1*q2)**2 + 0.25*(-1.0*q0**2 + 1.0*q1**2 - 1.0*q2**2 + 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) - g*q2 + 0.5*g*q3*(2*q0*q1 + 2*q2*q3)*(-1.0*q0**2 + 1.0*q1**2 - 1.0*q2**2 + 1.0*q3**2)/(sqrt((q0*q3 - q1*q2)**2 + 0.25*(-1.0*q0**2 + 1.0*q1**2 - 1.0*q2**2 + 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1))) + q1*(0.5*g*q1*(2*q0*q1 + 2*q2*q3)*(2.0*q0*q3 - 2.0*q1*q2)/(sqrt((q0*q3 - q1*q2)**2 + 0.25*(-1.0*q0**2 + 1.0*q1**2 - 1.0*q2**2 + 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + 0.5*g*q2*(2*q0*q1 + 2*q2*q3)*(-1.0*q0**2 + 1.0*q1**2 - 1.0*q2**2 + 1.0*q3**2)/(sqrt((q0*q3 - q1*q2)**2 + 0.25*(-1.0*q0**2 + 1.0*q1**2 - 1.0*q2**2 + 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + g*q3) - q2*(g*q0 - 0.5*g*q1*(2*q0*q1 + 2*q2*q3)*(-1.0*q0**2 + 1.0*q1**2 - 1.

In [28]:
# Check 60 degree bank to the right, given by q: [ 0.80010315  0.46193977 -0.19134172 -0.33141357]
# Should be 2g load factor
# Rate of turn should be: g/tas*tan(60) = g/tas*1.73
psi = 60*np.pi/180  
q60 = quaternion.Quaternion.axis_angle(np.array([0,0,1.0]), -np.pi/4)
q60 = q60*quaternion.Quaternion.axis_angle(np.array([0,1.0,0]), 10*np.pi/180)  # pitch down
q60 = q60*quaternion.Quaternion.axis_angle(np.array([1.0,0,0]), psi)
print("Euler angles (roll, pitch, heading):", q60.euler_angles()*180/np.pi)
q60 = q60.as_ndarray()
print("Accels:")
for n in range(4,7):
    tmp = statenext[n]
    for m in range(4):
        tmp = tmp.subs([q0,q1,q2,q3][m], q60[m])
    print(tmp)
    
print("Inertial rotation rates:")
q = quaternion.Quaternion(q0,q1,q2,q3)
w_inertial = (q*quaternion.Quaternion.from_vec(statenext[7:10])*q.inv()).as_ndarray()[1:]
for n in range(3):
    tmp = w_inertial[n]
    for m in range(4):
        tmp = tmp.subs([q0,q1,q2,q3][m], q60[m])
    print(tmp)

Euler angles (roll, pitch, heading): [60.0000000000000 10.0000000000000 -45.0000000000000]
Accels:
-0.664938106127251*g
-0.0514797425612272*g
1.88552569622526*g
Inertial rotation rates:
5.55111512312578e-17*g/tas
1.66533453693773e-16*g/tas
-1.73205080756888*g/tas


In [19]:
print(np.tan(60*np.pi/180))

1.7320508075688767
